In [1]:
import requests
import pandas as pd
from time import sleep
import datetime
from tqdm.notebook import tqdm

In [2]:
ibdata = pd.read_csv('true_isin_16909.csv',index_col=0)
ibdata=ibdata.dropna()
ibdata

,marketName,longName,isin
0,1ST,1ST GROUP LTD,AU0000001ST0
0,NMS,2U INC,US90214J1016
0,DDDX,3DX INDUSTRIES INC,US88556A1007
0,MMM,3M CO,US88579Y1010
0,88E,88 ENERGY LTD,AU00000088E2
...,...,...,...
0,ZYME,ZYMEWORKS INC,CA98985W1023
0,NMS,ZYNERBA PHARMACEUTICALS INC,US98986X1090
0,NMS,ZYNEX INC,US98986M1036
0,ZNGA,ZYNGA INC - CL A,US98986T1088


In [3]:
openfigi_apikey = ''#own apikey to increase limit
openfigi_url = 'https://api.openfigi.com/v2/mapping'
openfigi_headers = {'Content-Type': 'text/json'}

In [4]:
def map_jobs(jobs):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs)
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return response.json()

In [5]:
figi_data=pd.DataFrame(columns=['figi', 'name', 'ticker', 'exchCode', 'compositeFIGI', 'securityType',
       'marketSector', 'shareClassFIGI', 'securityType2',
       'securityDescription'])

In [6]:
empty= {'figi': '',
  'name': '',
  'ticker': '',
  'exchCode': '',
  'compositeFIGI': '',
  'securityType': '',
  'marketSector': '',
  'shareClassFIGI': '',
  'securityType2': '',
  'securityDescription': ''}

In [7]:
for i in tqdm(range(676)):
    isins = ibdata['isin'][25*i:25*i+25]
    jobs=[]
    for n in isins:
        newjob = {'idType': 'ID_ISIN', 'idValue': n},
        jobs.extend(newjob)
    job_results = map_jobs(jobs)
    just_dictionaries=[]
    for j in job_results:
        try:
            just_dictionaries.append(j['data'][0])
        except:
            just_dictionaries.append(empty)
    figi = pd.DataFrame.from_dict(just_dictionaries)
    figi_data=pd.concat([figi_data,figi])
    sleep(6)

  0%|          | 0/676 [00:00<?, ?it/s]

In [8]:
isins = ibdata['isin'][16900:]
jobs=[]
for n in isins:
    newjob = {'idType': 'ID_ISIN', 'idValue': n},
    jobs.extend(newjob)
job_results = map_jobs(jobs)
just_dictionaries=[]
for j in job_results:
    try:
        just_dictionaries.append(j['data'][0])
    except:
        just_dictionaries.append(empty)
figi = pd.DataFrame.from_dict(just_dictionaries)
figi_data=pd.concat([figi_data,figi])
sleep(6)

In [9]:
figi_data

,figi,name,ticker,exchCode,compositeFIGI,securityType,marketSector,shareClassFIGI,securityType2,securityDescription,uniqueID,uniqueIDFutOpt
0,BBG008HNLRF4,1ST GROUP LTD,1ST,AU,BBG008HNLRF4,Common Stock,Equity,BBG008HNLRG3,Common Stock,1ST,NaN,NaN
1,BBG001KS9450,2U INC,TWOU,US,BBG001KS9450,Common Stock,Equity,BBG001V0RR30,Common Stock,TWOU,NaN,NaN
2,BBG000C129N6,3DX INDUSTRIES INC,DDDX,US,BBG000C129N6,Common Stock,Equity,BBG001STRH79,Common Stock,DDDX,NaN,NaN
3,BBG000BP52R2,3M CO,MMM,US,BBG000BP52R2,Common Stock,Equity,BBG001S5T7X2,Common Stock,MMM,NaN,NaN
4,BBG000BWTTK9,88 ENERGY LTD,POQ,GR,BBG000BWTTK9,Common Stock,Equity,BBG001SCV1F7,Common Stock,POQ,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3,BBG00GD15LJ5,ZYMEWORKS INC,ZYME,US,BBG00GD15LJ5,Common Stock,Equity,BBG00247P1W0,Common Stock,ZYME,None,None
4,BBG007BBS8B7,ZYNERBA PHARMACEUTICALS INC,ZYNE,US,BBG007BBS8B7,Common Stock,Equity,BBG007BBS8C6,Common Stock,ZYNE,None,None
5,BBG000BJBXZ2,ZYNEX INC,ZYXI,US,BBG000BJBXZ2,Common Stock,Equity,BBG001S7T7V0,Common Stock,ZYXI,None,None
6,BBG005YZL376,ZYNGA INC - CL A,ZNGAEUR,XS,BBG005YZL2R6,Common Stock,Equity,BBG001T24RN2,Common Stock,ZNGAEUR,None,None


In [10]:
sum(figi_data.figi.isnull())

0

In [11]:
figi_data['new_index']=[i for i in range(len(figi_data))]
figi_data=figi_data.set_index(['new_index'])
figi_data

,figi,name,ticker,exchCode,compositeFIGI,securityType,marketSector,shareClassFIGI,securityType2,securityDescription,uniqueID,uniqueIDFutOpt
new_index,,,,,,,,,,,,
0,BBG008HNLRF4,1ST GROUP LTD,1ST,AU,BBG008HNLRF4,Common Stock,Equity,BBG008HNLRG3,Common Stock,1ST,NaN,NaN
1,BBG001KS9450,2U INC,TWOU,US,BBG001KS9450,Common Stock,Equity,BBG001V0RR30,Common Stock,TWOU,NaN,NaN
2,BBG000C129N6,3DX INDUSTRIES INC,DDDX,US,BBG000C129N6,Common Stock,Equity,BBG001STRH79,Common Stock,DDDX,NaN,NaN
3,BBG000BP52R2,3M CO,MMM,US,BBG000BP52R2,Common Stock,Equity,BBG001S5T7X2,Common Stock,MMM,NaN,NaN
4,BBG000BWTTK9,88 ENERGY LTD,POQ,GR,BBG000BWTTK9,Common Stock,Equity,BBG001SCV1F7,Common Stock,POQ,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16903,BBG00GD15LJ5,ZYMEWORKS INC,ZYME,US,BBG00GD15LJ5,Common Stock,Equity,BBG00247P1W0,Common Stock,ZYME,None,None
16904,BBG007BBS8B7,ZYNERBA PHARMACEUTICALS INC,ZYNE,US,BBG007BBS8B7,Common Stock,Equity,BBG007BBS8C6,Common Stock,ZYNE,None,None
16905,BBG000BJBXZ2,ZYNEX INC,ZYXI,US,BBG000BJBXZ2,Common Stock,Equity,BBG001S7T7V0,Common Stock,ZYXI,None,None


In [12]:
figi_data.to_csv('figi_data.csv')